In [6]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from pyngrok import ngrok

# Load Dataset
airline_data = pd.read_csv("us_airline_data.csv")
# Creating a Dash App
app=dash.Dash("Airline_Dashboard")

# App Layout
app.layout=html.Div(children=[html.H1("Airline Performance Dashboard",
                              style={"textAlign":"center",
                                     "color":"#503D36",
                                     "font-size":40
                                    }),
                              
                             html.Div(["Input Year: ", dcc.Input(id='input-year',
                                                                 value='2010',
                                                                 type='number',
                                                                 style={'height':'35px', 
                                                                        'font-size': 20}
                                                                )
                                      ],
                                      style={'font-size': 25}),
                              
                             html.Br(),
                             html.Br(),
                                 
                             html.Div([
                                 html.Div(dcc.Graph(id="line-plot")),
                                 html.Div(dcc.Graph(id="bar-plot"))],style={"display":"flex"}
                                     ),
                                 
                              html.Div([
                                  html.Div(dcc.Graph(id="Bubble-Chart")),
                                  html.Div(dcc.Graph(id="Pie Chart"))],style={"display":"flex"}
                                      )
                             ]
                   )

@app.callback([Output(component_id="line-plot", component_property="figure"),
                Output(component_id="bar-plot", component_property="figure"),
                Output(component_id="Bubble-Chart", component_property="figure"),
                Output(component_id="Pie Chart", component_property="figure")],
               Input (component_id="input-year", component_property="value")
             )

def compute_info(entered_year):
    df = airline_data[airline_data["Year"]==int(entered_year)]
    
    # Line Chart
    line_data = df.groupby('Month')['ArrDelay'].mean().reset_index()
    months={
        1: "January", 2: "February", 3: "March", 4: "April",5: "May", 6: "June", 7: "July",
        8: "August",  9: "September", 10: "October", 11: "November", 12: "December"}
    line_data["Month"]=line_data["Month"].replace(months)
    fig = go.Figure(data=go.Scatter(x=line_data['Month'], 
                                    y=line_data['ArrDelay'],
                                    mode='lines', 
                                    marker=dict(color='green')))
    fig.update_layout(title='Month vs Average Flight Delay Time',
                      xaxis_title='Month',
                      yaxis_title='ArrDelay')

    # Bar Chart
    bar_data =df.groupby("DestState")["Flights"].sum().reset_index()
    fig1=px.bar(bar_data,x="DestState",
                y="Flights",
                title="Total number of flights to the destination state split by reporting air",)

    # Bubble Chart
    bubble_data=df.groupby("Reporting_Airline")["Flights"].sum().reset_index()
    fig2=px.scatter(bubble_data,x="Reporting_Airline",
                    y="Flights",
                    title="Reporting Airline vs Number of Flights",
                    size="Flights",
                    hover_name="Reporting_Airline",
                    size_max=80)

    # Pie Chart
    pie_data=df.groupby("Month")["DistanceGroup"].sum().reset_index()
    months={
        1: "January", 2: "February", 3: "March", 4: "April",5: "May", 6: "June", 7: "July",
        8: "August",  9: "September", 10: "October", 11: "November", 12: "December"}
    pie_data["Month"]=pie_data["Month"].replace(months)
    fig3=px.pie(pie_data,values="DistanceGroup", 
               names="Month",
               title="Distance group proportion by month")
    fig3.update_layout(title_x=0.5)
    
    
    

    return fig, fig1, fig2, fig3

# Optional: Public URL via ngrok

public_url=ngrok.connect(8050)
print("Public URL:",public_url)
app.run()




Public URL: NgrokTunnel: "https://f5fd42ad7ed8.ngrok-free.app" -> "http://localhost:8050"
